In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

KeyboardInterrupt: 

In [ ]:
data = pd.read_csv('AutoData/train_auto.csv', error_bad_lines=False, encoding = "ISO-8859-1")
data2 = pd.read_csv('AutoData/test_auto.csv', error_bad_lines=False, encoding = "ISO-8859-1")

In [ ]:
data2.head()

In [ ]:
data2.shape

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.isna().sum(axis=0)

In [ ]:
data2.isna().sum(axis=0)

In [ ]:
data = data.dropna(thresh=data.shape[0]*0.5,how='all',axis=1)

In [ ]:
data = data.dropna(axis=0,how='any')
data.reset_index(inplace=True)

In [ ]:
data = data.drop(['index','INDEX'],axis=1)

In [ ]:
data.isna().sum(axis=0)

In [ ]:
data.head()

In [ ]:
data['PARENT1'] = data['PARENT1'].astype('category')
data['CAR_TYPE'] = data['CAR_TYPE'].astype('category')
data['RED_CAR'] = data['RED_CAR'].astype('category')
data['REVOKED'] = data['REVOKED'].astype('category')
data['URBANICITY'] = data['URBANICITY'].astype('category')
data['MSTATUS'] = data['MSTATUS'].astype('category')
data['SEX'] = data['SEX'].astype('category')
data['EDUCATION'] = data['EDUCATION'].astype('category')
data['JOB'] = data['JOB'].astype('category')
data['CAR_USE'] = data['CAR_USE'].astype('category')

In [ ]:
data['PARENT1'] = data['PARENT1'].cat.codes
data['CAR_TYPE'] = data['CAR_TYPE'].cat.codes
data['RED_CAR'] = data['RED_CAR'].cat.codes
data['REVOKED'] = data['REVOKED'].cat.codes
data['URBANICITY'] = data['URBANICITY'].cat.codes
data['MSTATUS'] = data['MSTATUS'].cat.codes
data['SEX'] = data['SEX'].cat.codes
data['EDUCATION'] = data['EDUCATION'].cat.codes
data['JOB'] = data['JOB'].cat.codes
data['CAR_USE'] = data['CAR_USE'].cat.codes

In [ ]:
data.head()

In [ ]:
for cols in ['INCOME','HOME_VAL','BLUEBOOK','OLDCLAIM']:
    for index, row in data.iterrows():
        data.loc[index, cols] = data.loc[index, cols].translate(str.maketrans('','','$,'))

In [ ]:
for cols in list(data.columns):
    data[cols] = pd.to_numeric(data[cols])

In [ ]:
list(data.columns)

In [ ]:
data.loc[0,:]

In [ ]:
correlation = data.corr(method='pearson')
columns = correlation.nlargest(20, 'TARGET_AMT').index
columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
correlation_map = np.corrcoef(data[data.columns].values.T)
sns.set(font_scale=1.0)
a4_dims = (20, 20)
fig, ax = plt.subplots(figsize=a4_dims)
heatmap = sns.heatmap(correlation_map,ax=ax, cbar=True, annot=True, square=True, yticklabels=data.columns.values, xticklabels=data.columns.values)
#plt.figure(figsize=(12,6))
plt.show()

In [ ]:
data.describe()

In [ ]:
df = data

In [ ]:
print(max(df['TARGET_AMT']))

In [ ]:
#Convert values for classification
for i in range(len(df['TARGET_AMT'])):
    if(df['TARGET_AMT'][i] <= 2000):
        df.loc[i,'TARGET_AMT'] = 2000
    elif(df['TARGET_AMT'][i] > 2000 and df['TARGET_AMT'][i] < 5000):
        df.loc[i,'TARGET_AMT'] = 2500
    elif(df['TARGET_AMT'][i] >= 5000 and df['TARGET_AMT'][i] < 10000):
        df.loc[i,'TARGET_AMT'] = 7500
    elif(df['TARGET_AMT'][i] >= 10000 and df['TARGET_AMT'][i] < 15000):
        df.loc[i,'TARGET_AMT'] = 12500
    elif(df['TARGET_AMT'][i] >= 15000 and df['TARGET_AMT'][i] < 20000):
        df.loc[i,'TARGET_AMT'] = 17500
    elif(df['TARGET_AMT'][i] >= 20000 and df['TARGET_AMT'][i] < 30000):
        df.loc[i,'TARGET_AMT'] = 25000
    elif(df['TARGET_AMT'][i] >= 30000 and df['TARGET_AMT'][i] < 50000):
        df.loc[i,'TARGET_AMT'] = 40000
    elif(df['TARGET_AMT'][i] >= 50000):
        df.loc[i,'TARGET_AMT'] = 50000
    

In [ ]:
print(df['TARGET_AMT'])

In [ ]:
"""#If with normalisation uncomment this and the next cell
x = data2withoutTextcols
y = x['INCURRED'].values
x = x.drop('INCURRED', axis = 1)"""
x = df
cols = x.columns
indices = x.index
y = x['TARGET_AMT']
Y = pd.DataFrame(y,columns=['TARGET_AMT'], index=indices)
x = x.drop(['TARGET_AMT','TARGET_FLAG'], axis = 1)
cols = x.columns
indices = x.index
df = pd.DataFrame(x,columns=cols, index=indices)

In [ ]:
len(x.columns)

In [ ]:
len(df.columns)

In [ ]:
"""
# Data Normalisation
from sklearn import preprocessing

cols = x.columns
indices = x.index

x = x.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
y = y.reshape(-1, 1)
Y = min_max_scaler.fit_transform(y)

df = pd.DataFrame(x_scaled,columns=cols, index=indices)
Y = pd.DataFrame(Y,columns=['INCURRED'], index=indices)"""

In [ ]:
df.isna().sum(axis=0)

In [ ]:
df.shape

In [ ]:
#df = df.dropna(axis=1,how="any")
df = df.dropna(axis=0,how="any")
df.reset_index(drop=True, inplace=True)
Y = Y.dropna(axis=0,how="any")
Y.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size = 0.25, random_state = 100)

In [ ]:
len(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

fig = plt.figure(figsize=(15,20))

def run_model(model, alg_name, plot_index):
    # build the model on training data
    model.fit(X_train, y_train)

    # make predictions for test data
    y_pred = model.predict(X_test)
    # calculate the accuracy score
    accuracy =  accuracy_score(y_test, y_pred) * 100
    
    acc = accuracy / 100
    # binomial confidence interval
    from math import sqrt
    interval = 1.96 * sqrt( (acc * (1 - acc)) / 1512)
    
    print('%.3f' % interval)
    print(alg_name,",accuracy:",accuracy)
    
    """# Compare the prediction result with ground truth
    color_code = {'Iris-virginica':'red', 'Iris-setosa':'blue', 'Iris-versicolor':'green'}

    # plt.figure(plot_index)
    ax = fig.add_subplot(4,2,plot_index) 
    colors = [color_code[x] for x in y_test.iloc[:,0]]
    ax.scatter(X_test.iloc[:,0], X_test.iloc[:,3], color=colors, marker='.', label='Circle = Ground truth')
    colors = [color_code[x] for x in y_pred]
    ax.scatter(X_test.iloc[:, 0], X_test.iloc[:,3], color=colors, marker='o', facecolors='none', label='Dot = Prediction')

    #plt.axes([0.65, 0.65, 0.2, 0.2])
    ax.legend(loc="lower right")
    # manually set legend color to black
    leg = plt.gca().get_legend()
    leg.legendHandles[0].set_color('black')
    leg.legendHandles[1].set_color('black')
    leg.legendHandles[1].set_facecolors('none')

    ax.set_title(alg_name + ". Accuracy: " + str(accuracy))"""

# ---- Decision Tree -----------
from sklearn import tree

model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)
run_model(model, "Decision Tree", 1)

# ----- Random Forest ---------------
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=10)
run_model(model, "Random Forest", 2)

# ----- xgboost ------------
# install xgboost
# 'pip install xgboost' or https://stackoverflow.com/questions/33749735/how-to-install-xgboost-package-in-python-windows-platform/39811079#39811079

"""from xgboost import XGBClassifier

model = XGBClassifier()
run_model(model, "XGBoost", 3)"""

# ------ SVM Classifier ----------------
from sklearn.svm import SVC
model = SVC()
run_model(model, "SVM Classifier", 4)

# -------- Nearest Neighbors ----------
from sklearn import neighbors
model = neighbors.KNeighborsClassifier()
run_model(model, "Nearest Neighbors Classifier", 5)

# ---------- SGD Classifier -----------------
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

model = OneVsRestClassifier(SGDClassifier())
run_model(model, "SGD Classifier", 6)

# --------- Gaussian Naive Bayes ---------
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
run_model(model, "Gaussian Naive Bayes", 7)

# ----------- Neural network - Multi-layer Perceptron  ------------
from sklearn.neural_network import MLPClassifier

model = MLPClassifier()
run_model(model, " MLP Neural network ", 8)


In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))

In [ ]:
pred = list(logreg.predict(X_test))
#print(list(X_test.index))
j = 0
for i in list(y_test.index):
    print(pred[j],y_test[i])
    j = j + 1
#pred[0]

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = logreg.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
pred = list(model.predict(X_test))
#print(list(X_test.index))
j = 0
for i in list(y_test.index):
    print(pred[j],y_test[i])
    j = j + 1
#pred[0]